In [1]:
import torch

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.5839, 0.2083, 0.5952],
        [0.9532, 0.3039, 0.0214],
        [0.4746, 0.4599, 0.8636],
        [0.6760, 0.6653, 0.8332],
        [0.1365, 0.0204, 0.3924]])


# Importing needed libraries

In [37]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
import argparse
import os
import matplotlib.image as mpimg
import torchvision, torch
from torchvision import datasets, models, transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from skimage import io, transform
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time
np.random.seed(0)
print('done')

done


In [35]:
print(torch.cuda.is_available())

True


# Modeling

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs=10
batch_size=32
learning_rate=0.001

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.resize = transforms.Resize((224, 224), antialias=True)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size= 5, stride=2) 
        self.conv2 = nn.Conv2d(24, 36, 5, 2) 
        self.conv3 = nn.Conv2d(36, 48, 5, 2) 
        self.conv4 = nn.Conv2d(48, 64, 3, 1)
        self.conv5 = nn.Conv2d(64, 64, 3, 1)   #img_size = 28224
        self.dropout = nn.Dropout(0.5)
        self.lin1 = nn.Linear(28224, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 2)
    def forward(self, out):
        out = out.reshape((-1, 3, 160, 320))
        out = self.resize(out)
        out = out.reshape((-1, 3, 160, 320))
        out = self.resize(out)
        out = F.elu(self.conv1(out))   
        out = F.elu(self.conv2(out))
        out = F.elu(self.conv3(out))
        out = F.elu(self.conv4(out))
        out = F.elu(self.conv5(out))
        out = self.dropout(out)
        out = out.view(out.size(0), -1) # This flattens the output of the conv layers
        out = F.elu(self.lin1(out))
        out = F.elu(self.lin2(out))
        out = F.elu(self.lin3(out))
        out = self.lin4(out)
        return out

model = ConvNet().to(device)
criterion=nn.MSELoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)
print("done")

done


# Dataloader

In [40]:
class cam (torch.utils.data.Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.frames = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.img_name= self.frames.iloc[:,0]
        self.transform = transform
        self.steering  = self.frames.iloc[:, 3]
        self.throttle  = self.frames.iloc[:, 4]

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, index):
        img=Image.open(os.path.join(self.root_dir,
                                self.img_name[index]))

        if self.transform:
            center_camera = self.transform(img)
        steering  = self.steering [index]
        throttle = self.throttle [index]
        return center_camera,steering ,throttle

# Data loading

In [41]:
transform_car = transforms.Compose([transforms.ToTensor()])
train_data = cam(csv_file="data\driving_log.csv",root_dir = "brbr\IMG",transform=transform_car )
train_loader=DataLoader(dataset=train_data,batch_size=32,drop_last=True,shuffle=True,num_workers=0)

In [42]:
valid_data =cam(csv_file="data_valid\driving_log.csv",root_dir = "valid\IMG",transform=transform_car )
valid_loader=DataLoader(dataset=valid_data,batch_size=32,shuffle=False,num_workers=0)

# Model Training

In [47]:
def training(model, num_epochs, train_loader, valid_loader, optimizer, device):
    start_time = time.time()
    minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []
    for epoch in range(num_epochs):

        model.train()
        for batch_idx, data in enumerate(train_loader):

            features,targets1,targets2 = data
            features = features.to(device)
            targets1 = targets1.to(device)
            targets2 = targets2.to(device)
            

            # ## FORWARD AND BACK PROP
            logits1,logits2= torch.split(model(features),1,dim =1) 
            loss1 = criterion(logits1, targets1.to(torch.float32))
            loss2 = criterion(logits2,targets2.to(torch.float32))
            loss = loss1 + loss2
            optimizer.zero_grad()

            loss.backward()

            # ## UPDATE MODEL PARAMETERS
            optimizer.step()

            # ## LOGGING
            minibatch_loss_list.append(loss.item())
            if not batch_idx % 50:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}')

        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for batch_idx_val, data_val in enumerate(valid_loader):

            features_val,targets1_val,targets2_val = data_val
            features_val = features_val.to(device)
            targets1_val = targets1_val.to(device)
            targets2_val = targets2_val.to(device)
            

            logits1_val,logits2_val = torch.split(model(features), 1,dim =1)
            loss1_val = criterion(logits1_val, targets1_val.to(torch.float32))
            loss2_val = criterion(logits2_val,targets2_val.to(torch.float32))
            valid_loss = loss1_val + loss2_val
            valid_acc_list.append(valid_loss.item())
            print(f'Epoch {epoch+1:03d}/{num_epochs:03d} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(model, 'saved_model.pth')

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    return minibatch_loss_list, valid_acc_list

In [48]:
minibatch_loss_list,valid_acc_list=training(model=model,
                                num_epochs=50,
                                train_loader=train_loader,
                                valid_loader=valid_loader,
                                optimizer=optimizer,
                                device=device)

C:\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([98, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 001/050 | Batch 0000/0033 | Loss: 0.9442
Epoch 001/050 		 Validation Loss: 0.002271240809932351
Validation Loss Decreased(8.000000--->0.077222) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.0006995725561864674
Validation Loss Decreased(0.077222--->0.023785) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.0006324133719317615
Validation Loss Decreased(0.023785--->0.021502) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.0008469264139421284
Epoch 001/050 		 Validation Loss: 0.0009916749550029635
Epoch 001/050 		 Validation Loss: 0.0005655488348565996
Validation Loss Decreased(0.021502--->0.019229) 	 Saving The Model
Epoch 001/050 		 Validation Loss: 0.001332747982814908
Epoch 001/050 		 Validation Loss: 0.000967244035564363
Epoch 001/050 		 Validation Loss: 0.0015836971579119563
Epoch 001/050 		 Validation Loss: 0.0006659431965090334
Epoch 001/050 		 Validation Loss: 0.000550534576177597
Validation Loss Decreased(0.019229--->0.018718) 	 Saving The Model
Epoch

C:\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([73])) that is different to the input size (torch.Size([98, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 001/050 		 Validation Loss: 0.0005698915338143706
Time elapsed: 0.67 min
Epoch: 002/050 | Batch 0000/0033 | Loss: 0.0268
Epoch 002/050 		 Validation Loss: 0.001248893910087645
Epoch 002/050 		 Validation Loss: 0.00012810739281121641
Validation Loss Decreased(0.016530--->0.004356) 	 Saving The Model
Epoch 002/050 		 Validation Loss: 9.194084123009816e-05
Validation Loss Decreased(0.004356--->0.003126) 	 Saving The Model
Epoch 002/050 		 Validation Loss: 0.00020715309074148536
Epoch 002/050 		 Validation Loss: 0.000308574759401381
Epoch 002/050 		 Validation Loss: 7.138228102121502e-05
Validation Loss Decreased(0.003126--->0.002427) 	 Saving The Model
Epoch 002/050 		 Validation Loss: 0.0005275660078041255
Epoch 002/050 		 Validation Loss: 0.0007969480357132852
Epoch 002/050 		 Validation Loss: 0.0006769720348529518
Epoch 002/050 		 Validation Loss: 9.375478839501739e-05
Epoch 002/050 		 Validation Loss: 3.506473149172962e-05
Validation Loss Decreased(0.002427--->0.001192) 	 Saving

Epoch: 006/050 | Batch 0000/0033 | Loss: 0.0238
Epoch 006/050 		 Validation Loss: 0.0012956070713698864
Epoch 006/050 		 Validation Loss: 0.00014511990593746305
Epoch 006/050 		 Validation Loss: 0.00010337615822209045
Epoch 006/050 		 Validation Loss: 0.00023645789769943804
Epoch 006/050 		 Validation Loss: 0.00034567638067528605
Epoch 006/050 		 Validation Loss: 7.448466931236908e-05
Epoch 006/050 		 Validation Loss: 0.0005866367137059569
Epoch 006/050 		 Validation Loss: 0.0007417688611894846
Epoch 006/050 		 Validation Loss: 0.0007543156971223652
Epoch 006/050 		 Validation Loss: 0.00011089747567893937
Epoch 006/050 		 Validation Loss: 4.200071998639032e-05
Epoch 006/050 		 Validation Loss: 0.0001752692915033549
Epoch 006/050 		 Validation Loss: 0.00034483251511119306
Epoch 006/050 		 Validation Loss: 4.9804060836322606e-05
Epoch 006/050 		 Validation Loss: 0.0006579949404112995
Epoch 006/050 		 Validation Loss: 0.0007354877307079732
Epoch 006/050 		 Validation Loss: 0.0058187670074

...

Epoch 030/050 		 Validation Loss: 0.0001605283614480868
Epoch 030/050 		 Validation Loss: 0.0003636008477769792
Time elapsed: 6.39 min
Epoch: 031/050 | Batch 0000/0033 | Loss: 0.0079
Epoch 031/050 		 Validation Loss: 0.0014619764406234026
Epoch 031/050 		 Validation Loss: 0.0001980910892598331
Epoch 031/050 		 Validation Loss: 0.0001458841870771721
Epoch 031/050 		 Validation Loss: 0.0003124899812974036
Epoch 031/050 		 Validation Loss: 0.0004363355983514339
Epoch 031/050 		 Validation Loss: 0.00010135969205293804
Epoch 031/050 		 Validation Loss: 0.0007185108261182904
Epoch 031/050 		 Validation Loss: 0.000659304903820157
Epoch 031/050 		 Validation Loss: 0.0009204711532220244
Epoch 031/050 		 Validation Loss: 0.00016411281831096858
Epoch 031/050 		 Validation Loss: 7.606780854985118e-05
Epoch 031/050 		 Validation Loss: 0.0002400961093371734
Epoch 031/050 		 Validation Loss: 0.00044835792505182326
Epoch 031/050 		 Validation Loss: 7.483414810849354e-05
Epoch 031/050 		 Validation Los

Epoch 035/050 		 Validation Loss: 9.784772555576637e-05
Epoch 035/050 		 Validation Loss: 0.0002264473441755399
Epoch 035/050 		 Validation Loss: 0.0003337102534715086
Epoch 035/050 		 Validation Loss: 7.10463136783801e-05
Epoch 035/050 		 Validation Loss: 0.0005691602709703147
Epoch 035/050 		 Validation Loss: 0.0007529488648287952
Epoch 035/050 		 Validation Loss: 0.0007322558667510748
Epoch 035/050 		 Validation Loss: 0.00010393748380010948
Epoch 035/050 		 Validation Loss: 3.760081381187774e-05
Epoch 035/050 		 Validation Loss: 0.00016675691585987806
Epoch 035/050 		 Validation Loss: 0.0003311462060082704
Epoch 035/050 		 Validation Loss: 4.661237835534848e-05
Epoch 035/050 		 Validation Loss: 0.0006391967181116343
Epoch 035/050 		 Validation Loss: 0.0007475339807569981
Epoch 035/050 		 Validation Loss: 0.005841563921421766
Epoch 035/050 		 Validation Loss: 0.0032619929406791925
Epoch 035/050 		 Validation Loss: 0.0007037676405161619
Epoch 035/050 		 Validation Loss: 0.000772406288

Epoch 039/050 		 Validation Loss: 0.0008173775277100503
Epoch 039/050 		 Validation Loss: 0.0001291585504077375
Epoch 039/050 		 Validation Loss: 5.2666098781628534e-05
Epoch 039/050 		 Validation Loss: 0.000198136389371939
Epoch 039/050 		 Validation Loss: 0.00038305047200992703
Epoch 039/050 		 Validation Loss: 5.688466626452282e-05
Epoch 039/050 		 Validation Loss: 0.0007113799219951034
Epoch 039/050 		 Validation Loss: 0.00069735775468871
Epoch 039/050 		 Validation Loss: 0.00587665056809783
Epoch 039/050 		 Validation Loss: 0.0032064293045550585
Epoch 039/050 		 Validation Loss: 0.0007166897994466126
Epoch 039/050 		 Validation Loss: 0.000743012351449579
Epoch 039/050 		 Validation Loss: 0.0001451823191018775
Epoch 039/050 		 Validation Loss: 0.00017422466771677136
Epoch 039/050 		 Validation Loss: 0.0003796736418735236
Epoch 039/050 		 Validation Loss: 8.727423119125888e-05
Epoch 039/050 		 Validation Loss: 9.895180846797302e-05
Epoch 039/050 		 Validation Loss: 0.000500316906254

Epoch 043/050 		 Validation Loss: 0.0005827359855175018
Epoch 043/050 		 Validation Loss: 0.0008064745925366879
Epoch 043/050 		 Validation Loss: 0.00585215212777257
Epoch 043/050 		 Validation Loss: 0.003343167481943965
Epoch 043/050 		 Validation Loss: 0.0007031980785541236
Epoch 043/050 		 Validation Loss: 0.0008117465185932815
Epoch 043/050 		 Validation Loss: 0.00018088592332787812
Epoch 043/050 		 Validation Loss: 0.00023112856433726847
Epoch 043/050 		 Validation Loss: 0.00047410346451215446
Epoch 043/050 		 Validation Loss: 0.00010941554501187056
Epoch 043/050 		 Validation Loss: 0.0001374049170408398
Epoch 043/050 		 Validation Loss: 0.0006143948994576931
Epoch 043/050 		 Validation Loss: 0.0006607602117583156
Epoch 043/050 		 Validation Loss: 0.0007792438846081495
Epoch 043/050 		 Validation Loss: 0.0002985245664604008
Epoch 043/050 		 Validation Loss: 0.0002177743153879419
Epoch 043/050 		 Validation Loss: 0.0004651887866202742
Epoch 043/050 		 Validation Loss: 0.00012738131

Epoch 047/050 		 Validation Loss: 0.00021783944976050407
Epoch 047/050 		 Validation Loss: 0.00027525523910298944
Epoch 047/050 		 Validation Loss: 0.0005309271509759128
Epoch 047/050 		 Validation Loss: 0.00014178022684063762
Epoch 047/050 		 Validation Loss: 0.00017528871831018478
Epoch 047/050 		 Validation Loss: 0.0006778665701858699
Epoch 047/050 		 Validation Loss: 0.0006920020678080618
Epoch 047/050 		 Validation Loss: 0.0008121709106490016
Epoch 047/050 		 Validation Loss: 0.0003447255294304341
Epoch 047/050 		 Validation Loss: 0.0002591187658254057
Epoch 047/050 		 Validation Loss: 0.0005230276728980243
Epoch 047/050 		 Validation Loss: 0.00016155277262441814
Epoch 047/050 		 Validation Loss: 0.0002029513707384467
Epoch 047/050 		 Validation Loss: 0.000434041052358225
Time elapsed: 9.62 min
Epoch: 048/050 | Batch 0000/0033 | Loss: 0.0204
Epoch 048/050 		 Validation Loss: 0.0013102315133437514
Epoch 048/050 		 Validation Loss: 0.00014202868624124676
Epoch 048/050 		 Validation 